In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gender_submission.csv', 'test.csv', 'train.csv']


In [90]:
df = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [91]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

In [92]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [93]:
df.drop(['Name'], axis=1, inplace = True)
df.drop(['PassengerId'], axis=1, inplace= True)
test.drop(['Name'], axis=1, inplace = True)


In [94]:
df.fillna(0, inplace = True)
test.fillna(0, inplace = True)


In [95]:
# Bins for ages
# Labels to categorize the ages.
bins = [0,1, 5, 10, 25, 50, 100]
labels = [1,2,3,4,5,6]

In [96]:
df['Age'] = pd.cut(df['Age'], bins = bins, labels = labels)
test['Age'] = pd.cut(test['Age'], bins = bins, labels = labels)


In [97]:
df = pd.get_dummies(df, columns = ['Sex'],drop_first = True )
test = pd.get_dummies(test, columns = ['Sex'],drop_first = True )

In [98]:
# Creating labels and bins for fare column
bins = [0,10,20,30,50, 100, 200 , 250, 300, 350, 400, 450, 500, 550]
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13]
df['Fare'] = pd.cut(df['Fare'], bins = bins, labels = labels)
test['Fare'] = pd.cut(test['Fare'], bins = bins, labels = labels)

In [99]:
label_enc = preprocessing.LabelEncoder()
df['Cabin'] = df['Cabin'].astype('str')
df['Cabin'] = label_enc.fit_transform(df['Cabin'])

test['Cabin'] = test['Cabin'].astype('str')
test['Cabin'] = label_enc.fit_transform(test['Cabin'])


In [100]:
df['Embarked'] = df['Embarked'].astype('str')
df['Embarked'] = label_enc.fit_transform(df['Embarked'])

test['Embarked'] = test['Embarked'].astype('str')
test['Embarked'] = label_enc.fit_transform(test['Embarked'])

In [101]:
df['Ticket'] = df['Ticket'].astype('str')
df['Ticket'] = label_enc.fit_transform(df['Ticket'])

test['Ticket'] = test['Ticket'].astype('str')
test['Ticket'] = label_enc.fit_transform(test['Ticket'])

In [102]:
df = df.astype('float')
df.fillna(0, inplace = True)

test = test.astype('float')
test.fillna(0, inplace = True)


In [103]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,3.603816,0.523008,0.381594,338.528620,2.579125,17.629630,2.529742,0.647587
std,0.486592,0.836071,2.000884,1.102743,0.806057,200.850657,1.794239,38.140335,0.800254,0.477990
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,3.000000,0.000000,0.000000,158.500000,1.000000,0.000000,2.000000,0.000000
50%,0.000000,3.000000,4.000000,0.000000,0.000000,337.000000,2.000000,0.000000,3.000000,1.000000
75%,1.000000,3.000000,5.000000,1.000000,0.000000,519.500000,4.000000,0.000000,3.000000,1.000000
max,1.000000,3.000000,6.000000,8.000000,6.000000,680.000000,13.000000,147.000000,3.000000,1.000000


In [104]:
X = np.array(df.drop(['Survived'], axis = 1))
X = preprocessing.scale(X)
# If we avoid this preprocessing then it will hurt a lot in K-Means clustering
Y = np.array(df['Survived'])

In [105]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,stratify = Y, test_size = 0.2, random_state = 31)

In [106]:
model = svm.SVC()

model.fit(X_train, Y_train)
predictions = model.predict(X_train)
accuracy = accuracy_score(predictions,Y_train)

print("Training accuracy = %0.2f" %(accuracy * 100))

Training accuracy = 84.55


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [107]:
predictions = model.predict(X_test)
accuracy = accuracy_score(predictions,Y_test)

print("Testing accuracy = %0.2f" %(accuracy * 100))

Testing accuracy = 79.33


In [108]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
0,892.0,3.0,5.0,0.0,0.0,152.0,1.0,0.0,1.0,1.0
1,893.0,3.0,5.0,1.0,0.0,221.0,1.0,0.0,2.0,0.0
2,894.0,2.0,6.0,0.0,0.0,73.0,1.0,0.0,1.0,1.0
3,895.0,3.0,5.0,0.0,0.0,147.0,1.0,0.0,2.0,1.0
4,896.0,3.0,4.0,1.0,1.0,138.0,2.0,0.0,2.0,0.0


In [109]:
Pid = test['PassengerId']
X_final_test = test.drop(['PassengerId'], axis = 1)
X_final_test = preprocessing.scale(X_final_test)


In [110]:
Y_pred = model.predict(X_final_test)

In [111]:
test['Survived'] = Y_pred

In [116]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male,Survived
0,892.0,3.0,5.0,0.0,0.0,152.0,1.0,0.0,1.0,1.0,0.0
1,893.0,3.0,5.0,1.0,0.0,221.0,1.0,0.0,2.0,0.0,0.0
2,894.0,2.0,6.0,0.0,0.0,73.0,1.0,0.0,1.0,1.0,0.0
3,895.0,3.0,5.0,0.0,0.0,147.0,1.0,0.0,2.0,1.0,0.0
4,896.0,3.0,4.0,1.0,1.0,138.0,2.0,0.0,2.0,0.0,1.0


In [113]:
data_to_submit = pd.DataFrame({
    'PassengerId':test['PassengerId'],
    'Survived':test['Survived']
})

In [117]:
data_to_submit.to_csv('csv_to_submit.csv', index = False)